# Most of this code should be pulled into seperate .py files

In [23]:
import pandas as pd
import numpy as np

from sklearn.decomposition import TruncatedSVD
import warnings
warnings.filterwarnings("ignore", category = RuntimeWarning)
from sklearn.metrics.pairwise import linear_kernel, euclidean_distances

from sklearn.preprocessing import normalize


In [2]:
# Training matrix
user_item = pd.DataFrame(
    pd.read_csv('/home/grimoire/Projects/BeerRatings/user_item_matrix.csv'))

X = user_item.values.T

In [3]:
# Test matrix
test_matrix = pd.DataFrame(
    pd.read_csv('/home/grimoire/Projects/BeerRatings/test_matrix.csv', index_col='review_profilename'))

test_matrix = test_matrix.T

In [4]:
beer_names = user_item.columns

In [5]:
def cosine_recommendations(beer, user_item=user_item) :
    
    # https://stackoverflow.com/questions/944592/best-practice-for-python-assert
    # Self note: build an exception class for beer parameter
    
    beer_names = user_item.columns
    assert beer in beer_names, 'The beer isn\'t currently in the recommendation list,\
 rate it more and it will be soon!' # This derives from user input not matching beer_names, add to exception class
    
    
    beer_name_list = list(beer_names) # Contains the index/name of the beer
    beer_index = beer_name_list.index(beer) # Finds the index of the beer parameter
    beer_list = \
    pd.DataFrame({'cosine_score':cosine_sim[beer_index]})\
    .sort_values(by='cosine_score', 
                 ascending=False)[1:8].index
    return list(beer_names[beer_list])

In [6]:
def pearson_recommendations(beer, user_item=user_item) :
    
    #############################################################
    # The coeffecient matrix is an 2D array of arrays
    # There is no indication of what columns refer to what beer
    # We will pull this information from our user_item matrix
    # Create a list of the beers from the columns
    # The index of the list will correspond with the coefficient matrix
    #############################################################
    
    # https://stackoverflow.com/questions/944592/best-practice-for-python-assert
    # Self note: build an exception class for beer parameter
    
    beer_names = user_item.columns
    assert beer in beer_names, 'The beer isn\'t currently in the recommendation list,\
 rate it more and it will be soon!' # This derives from user input not matching beer_names, add to exception class
    
    beer_name_list = list(beer_names) # Contains the index/name of the beer
    beer_index = beer_name_list.index(beer) # Finds the index of the beer parameter
    beer_list = \
    pd.DataFrame({'corrcoef_score':corr[beer_index]})\
    .sort_values(by='corrcoef_score', 
                 ascending=False)[1:8].index
    return list(beer_names[beer_list])

In [30]:
def euclidean_recommendations(beer, user_item=user_item) :
    
    #############################################################
    # The coeffecient matrix is an 2D array of arrays
    # There is no indication of what columns refer to what beer
    # We will pull this information from our user_item matrix
    # Create a list of the beers from the columns
    # The index of the list will correspond with the coefficient matrix
    #############################################################
    
    # https://stackoverflow.com/questions/944592/best-practice-for-python-assert
    # Self note: build an exception class for beer parameter
    
    beer_names = user_item.columns
    assert beer in beer_names, 'The beer isn\'t currently in the recommendation list,\
 rate it more and it will be soon!' # This derives from user input not matching beer_names, add to exception class
    
    beer_name_list = list(beer_names) # Contains the index/name of the beer
    beer_index = beer_name_list.index(beer) # Finds the index of the beer parameter
    beer_list = \
    pd.DataFrame({'corrcoef_score':eucl_sim[beer_index]})\
    .sort_values(by='corrcoef_score', 
                 ascending=False)[1:8].index
    return list(beer_names[beer_list])

In [16]:
svd = TruncatedSVD(n_components=24, random_state=3)
matrix = svd.fit_transform(X)

In [17]:
corr = np.corrcoef(matrix)
recommendation_precision = []
precision = 0
for name in test_matrix.columns : # name will be a column (profile name) from matrix
    user_list = list(test_matrix[test_matrix[name] > 0].index) # List of beers that the user drank
    benchmark = 0 # instantiating if any beers were drank from list of recommended
    for beer in user_list :
        recommended_beers = set(pearson_recommendations(beer))
        if recommended_beers & set(user_list) :
            benchmark += 1
    precision += benchmark / len(user_list)
recommendation_precision.append(precision / len(test_matrix.columns))

In [18]:
[recommendations for recommendations in recommendation_precision]

[0.01827364145917105]

In [19]:
cosine_sim = linear_kernel(matrix, matrix)
cosine_sim = normalize(cosine_sim)

In [20]:
recommendation_precision = []
precision = 0
for name in test_matrix.columns : # name will be a column (profile name) from matrix
    user_list = list(test_matrix[test_matrix[name] > 0].index) # List of beers that the user drank
    benchmark = 0 # instantiating if any beers were drank from list of recommended
    for beer in user_list :
        recommended_beers = set(cosine_recommendations(beer))
        if recommended_beers & set(user_list) :
            benchmark += 1
    precision += benchmark / len(user_list)
recommendation_precision.append(precision / len(test_matrix.columns))

In [21]:
[recommendations for recommendations in recommendation_precision]

[0.2537203281279971]

In [28]:
eucl_sim = euclidean_distances(matrix, matrix)

In [31]:
recommendation_precision = []
precision = 0
for name in test_matrix.columns : # name will be a column (profile name) from matrix
    user_list = list(test_matrix[test_matrix[name] > 0].index) # List of beers that the user drank
    benchmark = 0 # instantiating if any beers were drank from list of recommended
    for beer in user_list :
        recommended_beers = set(euclidean_recommendations(beer))
        if recommended_beers & set(user_list) :
            benchmark += 1
    precision += benchmark / len(user_list)
recommendation_precision.append(precision / len(test_matrix.columns))

In [32]:
[recommendations for recommendations in recommendation_precision]

[0.13841119966071316]